## Plane-DEM intersections

To test the routine for calculating the plane-DEM intersection, a few simulated topographic surfaces were used.

### Test case 1

The first test case is illustrated in the image below. We have a horizontal topographic surface, at a height of 0, with 100 x 100 cells with a cell size of 1. The geological plane dips 45° towards East. The source point for the plane is located at (0, 50, 50).

The locations of the expected intersection points are (50, *, 0).

![Test case 1](ims/inters_dem_plane_case_1.png)

First, a horizontal plane was created with Saga GIS and saved in *pygsf/example_data/horiz_plane.asc*.

Loading the dataset can be made with the following function:

In [1]:
from pygsf.libs_utils.gdal.gdal import try_read_raster_band

In [2]:
source_data = "/home/mauro/Documents/projects/gsf/pygsf/example_data/horiz_plane.asc"

In [3]:
success, cntnt = try_read_raster_band(raster_source=source_data)

In [4]:
print(success)

True


We read the data source with success. So we may unpack the result.

In [5]:
geotransform, projection, band_params, data = cntnt

In [6]:
type(geotransform)

pygsf.spatial.rasters.geotransform.GeoTransform

In [7]:
print(geotransform)

[  0.   1.   0. 100.   0.  -1.]


In [8]:
type(projection)

str

In [9]:
print(projection)

Hmmm, there is no projection info. In fact, there shouldn't..

In [10]:
type(band_params)

dict

A dictionary, as suspected. Try to see the content..

In [11]:
print(band_params)

{'unitType': '', 'noData': -99999.0, 'stats': {'max': 0.0, 'min': 0.0, 'std_dev': -1.0, 'mean': 0.0}, 'numColorTableEntries': 0, 'numOverviews': 0, 'dataType': 'Float32'}


A very horizontal surface, we agree..

In [12]:
type(data)

numpy.ndarray

In [13]:
data.shape

(100, 100)

In [14]:
data.min()

0.0

In [15]:
data.max()

0.0

Given these data, we store them into a GeoArray, a class imported from pygsf.spatial.rasters.geoarray:

In [16]:
from pygsf.spatial.rasters.geoarray import GeoArray

In [17]:
ga = GeoArray(inGeotransform=geotransform, inProjection=projection, inLevels=[data])

There is a single band provided in the geoarray, and represented by the data array.

The signature of the plane-DEM intersection function is:

**plane_dem_intersection** *(srcPlaneAttitude: Plane, srcPt: Point, geo_array: GeoArray, level_ndx: int=0) 
-> Tuple[List[Point], List[Point]]:*

We already have the geoarray, we need to define the source plane attitue and the source point.

The geoplane is East-dipping with a dip angle of 45°:

In [18]:
from pygsf.orientations.orientations import Plane
gplane = Plane(azim=90.0, dip_ang=45.0)

In [19]:
print(gplane)

Plane(090.00, +45.00)


The source point is located at (0, 50, 50)

In [20]:
from pygsf.spatial.vectorial.vectorial import Point
pt = Point(0, 50, 50)

Now we try calculating the intersection:

In [21]:
from pygsf.topography.plane_intersect import plane_dem_intersection
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:88: RuntimeWarning: divide by zero encountered in true_divide
  inters_residual_indices = np.where(abs(m_arr1 - m_arr2) < m_delta_tol, np.NaN, (q_arr2 - q_arr1) / (cell_size * (m_arr1 - m_arr2)))
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:97: RuntimeWarning: invalid value encountered in greater_equal
  inters_intracells_residuals = np.where(np.logical_and(inters_with_coincident_starts >= 0.0, inters_with_coincident_starts < 1.0), inters_with_coincident_starts, np.NaN)
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:97: RuntimeWarning: invalid value encountered in less
  inters_intracells_residuals = np.where(np.logical_and(inters_with_coincident_starts >= 0.0, inters_with_coincident_starts < 1.0), inters_with_coincident_starts, np.NaN)
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:222: RuntimeWarning: invalid value encountered in greater
  i

In [22]:
print(inters_pts)

[Point(50.0000, 99.5000, 0.0000), Point(50.0000, 98.5000, 0.0000), Point(50.0000, 97.5000, 0.0000), Point(50.0000, 96.5000, 0.0000), Point(50.0000, 95.5000, 0.0000), Point(50.0000, 94.5000, 0.0000), Point(50.0000, 93.5000, 0.0000), Point(50.0000, 92.5000, 0.0000), Point(50.0000, 91.5000, 0.0000), Point(50.0000, 90.5000, 0.0000), Point(50.0000, 89.5000, 0.0000), Point(50.0000, 88.5000, 0.0000), Point(50.0000, 87.5000, 0.0000), Point(50.0000, 86.5000, 0.0000), Point(50.0000, 85.5000, 0.0000), Point(50.0000, 84.5000, 0.0000), Point(50.0000, 83.5000, 0.0000), Point(50.0000, 82.5000, 0.0000), Point(50.0000, 81.5000, 0.0000), Point(50.0000, 80.5000, 0.0000), Point(50.0000, 79.5000, 0.0000), Point(50.0000, 78.5000, 0.0000), Point(50.0000, 77.5000, 0.0000), Point(50.0000, 76.5000, 0.0000), Point(50.0000, 75.5000, 0.0000), Point(50.0000, 74.5000, 0.0000), Point(50.0000, 73.5000, 0.0000), Point(50.0000, 72.5000, 0.0000), Point(50.0000, 71.5000, 0.0000), Point(50.0000, 70.5000, 0.0000), Point(50.

As expected, all the intersection points lie at (50, *, 0)

Plotting with Bokeh..

In [23]:
from bokeh.plotting import figure, output_notebook, show
x = list(map(lambda pt: pt.x, inters_pts))
y = list(map(lambda pt: pt.y, inters_pts))

output_notebook()
p = figure()
p.circle(x, y, size=2, color="navy", alpha=0.5)
show(p)

Loading BokehJS ...

### Test case 2

Now we consider a horizontal plane at z = 0 as topographic surface (same as case 1) and another horizontal surface at z = 1 as geological plane. We should get no intersection.

In [24]:
from pygsf.libs_utils.gdal.gdal import try_read_raster_band
source_data = "/home/mauro/Documents/projects/gsf/pygsf/example_data/horiz_plane.asc"
success, cntnt = try_read_raster_band(raster_source=source_data)

In [25]:
print(success)

True


In [26]:
geotransform, projection, band_params, data = cntnt
from pygsf.spatial.rasters.geoarray import GeoArray
ga = GeoArray(inGeotransform=geotransform, inProjection=projection, inLevels=[data])

The horizontal geological plane definition:

In [27]:
from pygsf.orientations.orientations import Plane
gplane = Plane(azim=90.0, dip_ang=0.0)

The source point located at (0, 50, 1)

In [28]:
from pygsf.spatial.vectorial.vectorial import Point
pt = Point(0, 50, 1)

In [29]:
from pygsf.topography.plane_intersect import plane_dem_intersection
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:88: RuntimeWarning: divide by zero encountered in true_divide
  inters_residual_indices = np.where(abs(m_arr1 - m_arr2) < m_delta_tol, np.NaN, (q_arr2 - q_arr1) / (cell_size * (m_arr1 - m_arr2)))
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:97: RuntimeWarning: invalid value encountered in greater_equal
  inters_intracells_residuals = np.where(np.logical_and(inters_with_coincident_starts >= 0.0, inters_with_coincident_starts < 1.0), inters_with_coincident_starts, np.NaN)
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:97: RuntimeWarning: invalid value encountered in less
  inters_intracells_residuals = np.where(np.logical_and(inters_with_coincident_starts >= 0.0, inters_with_coincident_starts < 1.0), inters_with_coincident_starts, np.NaN)
/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:222: RuntimeWarning: invalid value encountered in greater
  i

In [30]:
print(inters_pts)

[]


Ok, list is empty, as expected.

### Test case 3

Now we consider a horizontal plane at z = 0 as topographic surface (same as case 1) and another horizontal surface at z = 0 as geological plane. We should get all grid points as intersections.

The variables are the same as Case 2, apart from the point definition:

In [31]:
pt = Point(0, 50, 0)

In [32]:
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

/home/mauro/Documents/projects/gsf/pygsf/topography/plane_intersect.py:88: RuntimeWarning: invalid value encountered in true_divide
  inters_residual_indices = np.where(abs(m_arr1 - m_arr2) < m_delta_tol, np.NaN, (q_arr2 - q_arr1) / (cell_size * (m_arr1 - m_arr2)))


In [33]:
print(inters_pts)

[Point(0.5000, 99.5000, 0.0000), Point(1.5000, 99.5000, 0.0000), Point(2.5000, 99.5000, 0.0000), Point(3.5000, 99.5000, 0.0000), Point(4.5000, 99.5000, 0.0000), Point(5.5000, 99.5000, 0.0000), Point(6.5000, 99.5000, 0.0000), Point(7.5000, 99.5000, 0.0000), Point(8.5000, 99.5000, 0.0000), Point(9.5000, 99.5000, 0.0000), Point(10.5000, 99.5000, 0.0000), Point(11.5000, 99.5000, 0.0000), Point(12.5000, 99.5000, 0.0000), Point(13.5000, 99.5000, 0.0000), Point(14.5000, 99.5000, 0.0000), Point(15.5000, 99.5000, 0.0000), Point(16.5000, 99.5000, 0.0000), Point(17.5000, 99.5000, 0.0000), Point(18.5000, 99.5000, 0.0000), Point(19.5000, 99.5000, 0.0000), Point(20.5000, 99.5000, 0.0000), Point(21.5000, 99.5000, 0.0000), Point(22.5000, 99.5000, 0.0000), Point(23.5000, 99.5000, 0.0000), Point(24.5000, 99.5000, 0.0000), Point(25.5000, 99.5000, 0.0000), Point(26.5000, 99.5000, 0.0000), Point(27.5000, 99.5000, 0.0000), Point(28.5000, 99.5000, 0.0000), Point(29.5000, 99.5000, 0.0000), Point(30.5000, 99.5

They seem correct, just quite numerous..
We plot them with Bokeh.

In [34]:
from bokeh.plotting import figure, output_notebook, show
x = list(map(lambda pt: pt.x, inters_pts))
y = list(map(lambda pt: pt.y, inters_pts))

output_notebook()
p = figure()
p.circle(x, y, size=2, color="navy", alpha=0.5)
show(p)

Loading BokehJS ...

Looking at the Bokeh plot, we see that all cell centers have been plotted, as expected, since the topographic and the geological planes are coincident. So this is a "degenerate" case in which the intersection geometry is planar, not linear. 

###### Doc version: 2018-12-16